In [25]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

### Rail Service

In [2]:
rail_routes = (tbl.gtfs_schedule.routes()
               >> filter(_.route_type.isin(['0', '1', '2']))
               >> select(_.calitp_itp_id, _.route_id)
               >> collect())

In [3]:
def routes_to_stops(df_routes):
    df_routes['itp_id_route_id'] = df_routes.calitp_itp_id.astype(str) + df_routes.route_id
    
    df_trips = (tbl.gtfs_schedule.trips()
             >> filter(_.calitp_itp_id.isin(df_routes.calitp_itp_id))
             >> collect()
             >> select(_.calitp_itp_id, _.route_id, _.trip_id)
             >> filter(_.route_id.isin(df_routes.route_id)))
    
    df_trips['itp_id_route_id'] = df_trips.calitp_itp_id.astype(str) + df_trips.route_id
    df_trips.drop(columns=['calitp_itp_id', 'route_id'], inplace=True)
    
    df_trips = df_routes >> inner_join(_, df_trips, on='itp_id_route_id')
    df_trips['itp_id_trip_id'] = df_trips.calitp_itp_id.astype(str) + df_trips.trip_id
    df_operators = [int(str(x)) for x in list(df_trips.calitp_itp_id.unique())]
    
    df_stop_times = (tbl.gtfs_schedule.stop_times()
                  >> filter(_.calitp_itp_id.isin(df_operators))
                  >> select(_.calitp_itp_id, _.stop_id, _.trip_id)
                  >> collect()
                    )
    df_stop_times['itp_id_trip_id'] = df_stop_times.calitp_itp_id.astype(str) + df_stop_times.trip_id
    df_stop_times.drop(columns=['calitp_itp_id', 'trip_id'], inplace=True)
    df_stop_times = df_stop_times >> inner_join(_, df_trips, on='itp_id_trip_id')
    df_stop_times = df_stop_times.drop_duplicates(subset=['stop_id', 'calitp_itp_id'])
    df_stop_times['itp_id_stop_id'] = df_stop_times.calitp_itp_id.astype(str) + df_stop_times.stop_id
    
    df_stops = (tbl.gtfs_schedule.stops()
              >> select(_.stop_id, _.calitp_itp_id, _.stop_lat, _.stop_lon)
              >> filter(_.calitp_itp_id.isin(df_stop_times.calitp_itp_id))
              >> collect()
             )
    
    df_stops['itp_id_stop_id'] = df_stops.calitp_itp_id.astype(str) + df_stops.stop_id
    df_stop_times.drop(columns=['calitp_itp_id', 'stop_id'], inplace=True)
    df_stops = df_stops >> inner_join(_, df_stop_times, on='itp_id_stop_id')
    df_stops = gpd.GeoDataFrame(df_stops,
                              geometry = gpd.points_from_xy(df_stops.stop_lon, df_stops.stop_lat),
                              crs = 'EPSG:4326')
    return df_stops.to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)

In [4]:
def map_hqta(gdf, mouseover=None):
    global nix_list
    nix_list = []
    
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
    gdf = gdf.to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    if gdf.geometry.iloc[0].geom_type == 'Point':
        gdf.geometry = gdf.geometry.buffer(200)
    
    x = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.x
    y = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover}')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{}</b></h3>
            '''.format(feature['properties'][mouseover])
            
        def add_to_nix(feature, **kwargs):
            nix_list.append(feature['properties'][mouseover])
            
    if 'hq_transit_corr' in gdf.columns:
        geo_data_hq = GeoData(geo_dataframe = gdf[gdf['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#08589e',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'HQTA')
        #a8ddb5
        geo_data_not_hq = GeoData(geo_dataframe = gdf[~gdf['hq_transit_corr']].to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#fec44f',
                                            'opacity':0.2, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'non-HQTA')

        m.add_layer(geo_data_hq)
        m.add_layer(geo_data_not_hq)
    
    else:
    
        geo_data_hq = GeoData(geo_dataframe = gdf.to_crs('EPSG:4326'),
                               style={'color': 'black', 'fillColor': '#08589e',
                                            'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                               hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                               name = 'gdf')
        m.add_layer(geo_data_hq)
    
    if mouseover:
        geo_data_hq.on_hover(update_html)
        geo_data_hq.on_hover(add_to_nix)

    m.add_control(LayersControl())

    return m

In [5]:
rail_stops = routes_to_stops(rail_routes)

In [26]:
map_hqta(rail_stops)

Map(center=[37.558559005208195, -122.007598], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

#### BRT Service likely meeting [PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit) definition:

* LA Metro Orange, Silver excluding street running (stop flags only)
* Omnitrans sbX, all stops (curbside stations are well-defined, with fare prepayment)
* AC Transit Tempo, all stops (curbside stations are well-defined, with fare prepayment)

In [7]:
metro_brt = (tbl.gtfs_schedule.routes()
         >> filter(_.calitp_itp_id == 182)
         >> filter(_.route_id.isin(['901-13149', '910-13149']))
         >> collect())

In [8]:
metro_brt_stops = routes_to_stops(metro_brt)

In [9]:
## unable to filter out non-station stops using GTFS, manual list:
metro_street_running =['141012', '13805', '5397', '13803',
 '13804', '5396', '13802', '5395', '5410', '5411', '13817',
 '12304', '5408', '3821', '2603', '3153', '3124', '378', '65300039',
 '65300038', '15820', '13460', '4994', '1813', '2378', '5049',
 '4652', '2377', '4675', '5040', '65300042', '3674', '15713',
 '13561', '5378', '13560', '70500012', '5377', '15612',
 '12416', '11917', '12415', '8704']

In [10]:
metro_brt_stops = metro_brt_stops >> filter(-_.stop_id.isin(metro_street_running))

In [11]:
map_hqta(metro_brt_stops)

Map(center=[34.180897003816504, -118.46902599999993], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [12]:
act_brt = (tbl.gtfs_schedule.routes()
         >> filter(_.calitp_itp_id == 4)
         >> filter(_.route_id == '1T')
         >> collect())

In [13]:
act_brt_stops = routes_to_stops(act_brt)

In [14]:
map_hqta(act_brt_stops)

Map(center=[37.78878500530988, -122.24642099999997], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [15]:
omni_brt = (tbl.gtfs_schedule.routes()
         >> filter(_.calitp_itp_id == 232)
         >> filter(_.route_id == '9648')
         >> collect())

In [16]:
omni_brt_stops = routes_to_stops(omni_brt)

In [17]:
map_hqta(omni_brt_stops)

Map(center=[34.0486850037653, -117.25235699999993], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Ferry

In [18]:
ferry = (tbl.gtfs_schedule.routes()
         >> filter(_.route_type == '4')
         # >> filter(_.route_id == '9648')
         >> collect())

In [19]:
ferry_stops = routes_to_stops(ferry)
angel_and_alcatraz = ['2483552', '2483550'] ##only stops without bus service, TODO implement algorithm
ferry_stops = ferry_stops >> filter(-_.stop_id.isin(angel_and_alcatraz))

In [20]:
map_hqta(ferry_stops)

Map(center=[37.795497005312825, -122.39336499999999], controls=(ZoomControl(options=['position', 'zoom_in_text…

## Combined

In [21]:
rail_brt_ferry = (rail_stops
                 .append(metro_brt_stops)
                 .append(act_brt_stops)
                 .append(omni_brt_stops)
                 .append(ferry_stops))

In [22]:
rail_brt_ferry

,stop_id,calitp_itp_id,stop_lat,stop_lon,itp_id_stop_id,itp_id_trip_id,route_id,itp_id_route_id,trip_id,geometry,...,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color,route_sort_order,continuous_pickup,continuous_drop_off,calitp_extracted_at
0,3400005,10,37.558559,-122.007598,103400005,10ACE 06,ACE,10ACE,ACE 06,POINT (-177088.231 -49017.725),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3400004.1,10,37.697081,-121.717648,103400004.1,10ACE 06,ACE,10ACE,ACE 06,POINT (-151236.096 -34123.815),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3400006.3,10,37.329568,-121.903188,103400006.3,10ACE 06,ACE,10ACE,ACE 06,POINT (-168390.559 -74658.031),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3400006.1,10,37.406398,-121.966671,103400006.1,10ACE 06,ACE,10ACE,ACE 06,POINT (-173829.474 -66004.617),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3400001,10,37.957058,-121.278948,103400001,10ACE 06,ACE,10ACE,ACE 06,POINT (-112222.169 -5836.078),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,890002,2,37.794360,-122.390790,2890002,2t_1063999_b_26132_tn_0,SF - Ala - Oak,2SF - Ala - Oak,t_1063999_b_26132_tn_0,POINT (-210221.680 -22028.923),...,SF - Ala - Oak,San Francisco - Alameda via Oakland,None,None,ec7501,None,None,None,None,2021-04-16
74,890001,2,37.795106,-122.391612,2890001,2t_1216480_b_26809_tn_0,Rich - SF,2Rich - SF,t_1216480_b_26809_tn_0,POINT (-210291.852 -21944.196),...,Rich - SF,Richmond - San Francisco,None,None,007e12,ffffff,None,None,None,2021-04-16
75,890001,2,37.795106,-122.391612,2890001,2t_1216480_b_26809_tn_0,Rich - SF,2Rich - SF,t_1216480_b_26809_tn_0,POINT (-210291.852 -21944.196),...,Rich - SF,Richmond - San Francisco,None,None,007e12,ffffff,None,None,None,2021-04-16
76,12030042,2,37.791153,-122.294157,212030042,2t_1063999_b_26132_tn_0,SF - Ala - Oak,2SF - Ala - Oak,t_1063999_b_26132_tn_0,POINT (-201735.070 -22595.234),...,SF - Ala - Oak,San Francisco - Alameda via Oakland,None,None,ec7501,None,None,None,None,2021-04-16


In [23]:
map_hqta(rail_brt_ferry)

Map(center=[37.558559005208195, -122.007598], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

In [24]:
rail_brt_ferry.to_parquet('./data/rail_ferry_brt/rail_brt_ferry.parquet')

/tmp/ipykernel_10665/2291923242.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  rail_brt_ferry.to_parquet('./data/rail_ferry_brt/rail_brt_ferry.parquet')
